In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 500

In [2]:
#Training Data
DATAPATH = 'dfs/final_transformed_data.csv' 

df_original = pd.read_csv(DATAPATH)
#df = df[~df.overview.isna()]
print(len(df_original))
print(df_original.info())
print(df_original.head(5))

13344
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13344 entries, 0 to 13343
Data columns (total 86 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   supplier_sample_pulls__created_at               13344 non-null  int64  
 1   supplier_sample_pulls__sampling_method          13344 non-null  object 
 2   sample_pulls__target_groups                     13344 non-null  object 
 3   sample_pulls__country                           13344 non-null  object 
 4   sample_pulls__language                          13344 non-null  object 
 5   sample_pulls__cost_per_interview                13344 non-null  float64
 6   sample_pulls__incident_rate                     13344 non-null  float64
 7   sample_pulls__length_of_interview               13344 non-null  int64  
 8   sample_pulls__completes_needed                  13344 non-null  int64  
 9   sample_pulls__start_date         

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (80,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#assign two sentence 
#1 for continent-lang-study combine column
#2 for targetgroup qualification column
df = df_original
df.rename(columns={'continent_language_study_combine':'sentence_1'}, inplace=True)
df.rename(columns={'projects__target_groups_qualifications_combine':'sentence_2'}, inplace=True)

In [4]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

#STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=None, lemmatize=False):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    
    tokens = [w for w in word_tokenize(sentence)]
    print(tokens)
    '''
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    '''
    return tokens
    
'''    
#Used in Word2vec
def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    #df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=None, lemmatize=True))
    return df
'''

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\junaid.shaikh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'    \n#Used in Word2vec\ndef clean_sentences(df):\n    """\n    Remove irrelavant characters (in new column clean_sentence).\n    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).\n    """\n    print(\'Cleaning sentences...\')\n    #df[\'clean_sentence\'] = df[\'sentence\'].apply(clean_text)\n    df[\'tok_lem_sentence\'] = df[\'sentence\'].apply(\n        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=None, lemmatize=True))\n    return df\n'

In [5]:
#save tfidf model
import pickle as pk

def save_embedding(model, name):
    with open(f'model/tfidf_model_{name}.pkl', 'wb') as fn:
        pk.dump(model, fn)

def save_vectorizer(vectorizer, name):
    with open(f'model/tfidf_{name}.pkl', 'wb') as fn:
        pk.dump(vectorizer, fn)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Adapt stop words
#token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)

# Fit TFIDF
#vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer)
vectorizer_1 = TfidfVectorizer(stop_words=None, tokenizer=tokenizer)
vectorizer_2 = TfidfVectorizer(stop_words=None, tokenizer=tokenizer)
tfidf_mat_1 = vectorizer_1.fit_transform(df['sentence_1'].values) # -> (num_sentences, num_vocabulary)
save_tfidf_embedding(tfidf_mat_1, 'sentence_1')
save_vectorizer(vectorizer_1, 'vectorizer_1')

['europe', 'eng', 'healthcare']
['europe', 'eng', 'healthcare']
['europe', 'eng', 'healthcare']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'healthcare']
['north', 'america', 'eng', 'healthcare']
['north', 'america', 'eng', 'healthcare']
['north', 'america', 'eng', 'consumer', 'study']
['north', 'america', 'eng', 'consumer', 'study']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'consumer', 'study']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
['north', 'america', 'eng', 'b2b']
[

In [ ]:
tfidf_mat_2 = vectorizer_2.fit_transform(df['sentence_2'].values) # -> (num_sentences, num_vocabulary)
save_embedding(tfidf_mat_2, 'sentence_2')
save_vectorizer(vectorizer_2, 'vectorizer_2')

In [ ]:
import spacy

# !python -m spacy download en_core_web_lg

#Load pre-trained model
nlp = spacy.load("en_core_web_lg") 
# Apply the model to the sentences
df['spacy_sentence_1'] = df['sentence_1'].apply(lambda x: nlp(x))
df['spacy_sentence_2'] = df['sentence_2'].apply(lambda x: nlp(x)) 
# Retrieve the embedded vectors as a matrix 
embed_mat_1 = df['spacy_sentence_1'].values
embed_mat_2 = df['spacy_sentence_2'].values
embed_mat_1.shape
embed_mat_2.shape

In [ ]:
def predict_spacy(model, query_sentence, embed_mat, topk=10):
    """
    Predict the topk sentences after applying spacy model.
    """
    query_embed = model(query_sentence)
    mat = np.array([query_embed.similarity(line) for line in embed_mat])
    # keep if vector has a norm
    mat_mask = np.array(
        [True if line.vector_norm else False for line in embed_mat])
    best_index_2 = extract_best_indices(mat, topk=topk, mask=mat_mask)
    return best_index_2

In [ ]:
#call INFERENCE transformation method for getting 
#query_sentence_1
#query_sentence_2

In [ ]:
#Calculate match-prediction
best_spacy_index_1 = predict_spacy(nlp, query_sentence_1, embed_mat_1)
best_spacy_index_2 = predict_spacy(nlp, query_sentence_2, embed_mat_2)


In [ ]:
#Show predictions
display(df[['rl','projects__name','sentence_1']].iloc[best_spacy_index_1])
display(df[['rl','projects__name','sentence_2']].iloc[best_spacy_index_2])

In [ ]:
#WORD2VEC
df_sentence_1 = df[['rl','projects__name','sentence_1']]
df_sentence_2 = df[['rl','projects__name','sentence_2']]

#Used for Word2Vec model
df_sentence_1_cleaned = clean_sentences(df_sentence_1)
df_sentence_2_cleaned = clean_sentences(df_sentence_2)